In [ ]:
from transformers import AutoTokenizer, GPTNeoXForCausalLM, GPTNeoXConfig
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import torch
import os

tqdm.pandas()

In [ ]:
class HFMemoriesDataset(Dataset):
    is_dataframe = False

    def __init__(self, memories, tokenizer, sample=None):
        self.tokenizer = tokenizer
        self.memories = memories
        if sample is not None:
            self.memories = self.memories.to_pandas().sample(sample)
            self.is_dataframe = True

    def __getitem__(self, index):
        memory_record = (
            self.memories.iloc[index] if self.is_dataframe else self.memories[index]
        )
        decoded_text = self.tokenizer.decode(memory_record["tokens"])
        return decoded_text

    def __len__(self):
        return len(self.memories)


def load_tokenizer(split_name):
    isDeduped = split_name.startswith("deduped")
    model = split_name.split("duped.")[-1]
    corresponding_model = f"EleutherAI/pythia-{model}{'-deduped' if isDeduped else ''}"
    tokenizer =  AutoTokenizer.from_pretrained(corresponding_model)
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

def load_model(split_name):
    isDeduped = split_name.startswith("deduped")
    model = split_name.split("duped.")[-1]
    corresponding_model = f"EleutherAI/pythia-{model}{'-deduped' if isDeduped else ''}"
    device = torch.device("cuda:7")
    return GPTNeoXForCausalLM.from_pretrained(corresponding_model, device_map="auto", offload_folder="offload").eval()

In [ ]:
split_name = "deduped.12b"
memories = load_dataset("EleutherAI/pythia-memorized-evals")[split_name]
tokenizer = load_tokenizer(split_name)
memories_dataset = HFMemoriesDataset(
    load_dataset("EleutherAI/pythia-memorized-evals")[split_name], 
    tokenizer)

pythia_model = load_model(split_name)

In [ ]:
def calculate_perplexity(logits, labels):
    shift_logits = logits.detach()[:-1, :].contiguous()
    shift_labels = labels[1:].contiguous()
    loss_fct = torch.nn.CrossEntropyLoss()
    cross_entropy = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    perplexity = torch.exp(cross_entropy)
    return perplexity

data_loader = DataLoader(memories_dataset, batch_size=32)
all_perplexities = []

with torch.no_grad():
    for batch in tqdm(data_loader):
        tokenized_batch = tokenizer(
            batch, return_tensors="pt", max_length=512, truncation=True, padding=True
        )
        tokenized_batch.to(torch.device("cuda:7"))
        labels = tokenized_batch["input_ids"][:, 1:].contiguous()

        outputs = pythia_model(**tokenized_batch, labels=tokenized_batch["input_ids"])
        logits = outputs.logits.detach()

        labels = tokenized_batch["input_ids"]
        perplexities = [calculate_perplexity(logits[i], labels[i]) for i in range(len(logits))]
        all_perplexities += [perplexity.item() for perplexity in perplexities]

print(len(all_perplexities))
all_perplexities

In [ ]:
memories_df = memories.to_pandas()[["index"]]
memories_df["perplexity"] = all_perplexities
memories_df[["index", "perplexity"]].to_csv(f"./datasets/memories_{split_name}.csv", index=False)
memories_df